In [ ]:
import pandas as pd
data=#add dataset path
df=pd.read_csv(data)
df.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [ ]:
!pip install transformers

In [ ]:
!pip install transformers dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.41
    Uninstalling SQLAlchemy-2.0.41:
      Successfully uninstalled SQLAlchemy-2.0.41
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer

In [ ]:
print("Total rows and columns:", df.shape)
print("Missing Values:",df.isnull().sum())
print("Number of duplicate rows:", df.duplicated().sum())
print("Number of unique rows:", len(df.drop_duplicates()))
print("Datatypes:",df.dtypes)
print("First Five:",df.head())

Total rows and columns: (40432, 6)
Missing Values: category         0
rating           0
label            0
text_            0
cleaned_text     0
encoded_label    0
dtype: int64
Number of duplicate rows: 12
Number of unique rows: 40420
Datatypes: category          object
rating           float64
label              int64
text_             object
cleaned_text      object
encoded_label      int64
dtype: object
First Five:              category  rating  label  \
0  Home_and_Kitchen_5     5.0      0   
1  Home_and_Kitchen_5     5.0      0   
2  Home_and_Kitchen_5     5.0      0   
3  Home_and_Kitchen_5     1.0      0   
4  Home_and_Kitchen_5     5.0      0   

                                               text_  \
0  Love this!  Well made, sturdy, and very comfor...   
1  love it, a great upgrade from the original.  I...   
2  This pillow saved my back. I love the look and...   
3  Missing information on how to use it, but it i...   
4  Very nice set. Good quality. We have had the s...   


In [ ]:
!pip install tensorflow pandas scikit-learn matplotlib

In [ ]:
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|[^a-zA-Z\s]", "", text)
    return text
df['cleaned_text'] = df['text_'].apply(clean_text)
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['label'])
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['cleaned_text'], df['encoded_label'], test_size=0.2, stratify=df['encoded_label'], random_state=42
)

In [ ]:
df.head()

,category,rating,label,text_,cleaned_text,encoded_label
0,Home_and_Kitchen_5,5.0,0,"Love this! Well made, sturdy, and very comfor...",love this well made sturdy and very comfortab...,0
1,Home_and_Kitchen_5,5.0,0,"love it, a great upgrade from the original. I...",love it a great upgrade from the original ive...,0
2,Home_and_Kitchen_5,5.0,0,This pillow saved my back. I love the look and...,this pillow saved my back i love the look and ...,0
3,Home_and_Kitchen_5,1.0,0,"Missing information on how to use it, but it i...",missing information on how to use it but it is...,0
4,Home_and_Kitchen_5,5.0,0,Very nice set. Good quality. We have had the s...,very nice set good quality we have had the set...,0


In [ ]:
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_texts)
x_train = tokenizer.texts_to_sequences(train_texts)
x_test = tokenizer.texts_to_sequences(test_texts)
x_train = pad_sequences(x_train, maxlen=max_len, padding='post')
x_test = pad_sequences(x_test, maxlen=max_len, padding='post')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model = Sequential([
    Embedding(input_dim=max_words, output_dim=64),

    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train, train_labels, validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
809/809 ━━━━━━━━━━━━━━━━━━━━ 196s 230ms/step - accuracy: 0.7857 - loss: 0.4000 - val_accuracy: 0.9461 - val_loss: 0.1476
Epoch 2/5
809/809 ━━━━━━━━━━━━━━━━━━━━ 187s 232ms/step - accuracy: 0.9613 - loss: 0.1116 - val_accuracy: 0.9470 - val_loss: 0.1368
Epoch 3/5
809/809 ━━━━━━━━━━━━━━━━━━━━ 195s 223ms/step - accuracy: 0.9774 - loss: 0.0661 - val_accuracy: 0.9501 - val_loss: 0.1539
Epoch 4/5
809/809 ━━━━━━━━━━━━━━━━━━━━ 202s 223ms/step - accuracy: 0.9856 - loss: 0.0442 - val_accuracy: 0.9519 - val_loss: 0.1527
Epoch 5/5
809/809 ━━━━━━━━━━━━━━━━━━━━ 203s 225ms/step - accuracy: 0.9870 - loss: 0.0366 - val_accuracy: 0.9508 - val_loss: 0.2056


In [ ]:
from sklearn.metrics import classification_report
target_names = [str(cls) for cls in label_encoder.classes_]
preds = model.predict(x_test)
pred_labels = (preds > 0.5).astype("int32")
print(classification_report(test_labels, pred_labels, target_names=target_names))

253/253 ━━━━━━━━━━━━━━━━━━━━ 14s 53ms/step
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      4044
           1       0.95      0.96      0.95      4043

    accuracy                           0.95      8087
   macro avg       0.95      0.95      0.95      8087
weighted avg       0.95      0.95      0.95      8087



In [ ]:
loss, accuracy = model.evaluate(x_test, test_labels)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

253/253 ━━━━━━━━━━━━━━━━━━━━ 14s 53ms/step - accuracy: 0.9502 - loss: 0.2387
Model Accuracy: 95.21%


In [ ]:
model.save('review_classifier.keras')

In [ ]:
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import re
model = load_model('review_classifier.keras')
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|[^a-zA-Z\s]", "", text)
    return text.strip()
def predict_review_class(review_text, max_len=100):
    cleaned = clean_text(review_text)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    pred = model.predict(padded)[0][0]
    label = "OR Review" if pred > 0.5 else "CG Review"
    confidence = pred if pred > 0.5 else 1 - pred
    return f"Prediction: {label} (Confidence: {confidence:.2f})"
print(predict_review_class("Great quality and delivery was fast!"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step
Prediction: OR Review (Confidence: 0.91)
